In [ ]:
import numpy as np
import os
import cv2
from random import shuffle
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_sample_weight

# Function to load and preprocess the data with enhanced data augmentation
def create_data(folder_path, target_size=(224, 224), batch_size=32):
    datagen = ImageDataGenerator(
        rotation_range=45,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    data = []
    labels_mapping = {}  # to map string labels to numerical values
    label_count = 0

    for folder in tqdm(os.listdir(folder_path)):
        labels_mapping[folder] = label_count
        label_count += 1
        for img in os.listdir(os.path.join(folder_path, folder)):
            try:
                path = os.path.join(folder_path, folder, img)
                img_data = cv2.imread(path, cv2.IMREAD_COLOR)

                # Check if the loaded image data is not empty
                if img_data is not None:
                    img_data = cv2.resize(img_data, target_size)
                    img_data = preprocess_input(img_data)  # Use MobileNetV3 preprocessing
                    data.append([np.array(img_data), folder])
            except Exception as e:
                print(f"Error processing {path}: {str(e)}")

    shuffle(data)
    return datagen, data, labels_mapping

# Load and preprocess the data (224x224 resolution with enhanced data augmentation)
folder_path = '/kaggle/input/bangla-handwritten-digit/total image 1-50'
datagen, data, labels_mapping = create_data(folder_path, target_size=(224, 224))
X = np.array([i[0] for i in data])
y = np.array([labels_mapping[i[1]] for i in data])

# Save labels mapping to a text file
labels_file_path = '/kaggle/working/labels.txt'
with open(labels_file_path, 'w') as f:
    for label, value in labels_mapping.items():
        f.write(f"{label}: {value}\n")

print(f'Labels mapping saved to: {labels_file_path}')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load pre-trained MobileNetV3Large model as the feature extractor
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(len(labels_mapping), activation='softmax'))  # Adjust output layer size

model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

class_weights = compute_sample_weight('balanced', y_train)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training the model
model.fit(datagen.flow(X_train, y_train, sample_weight=class_weights, batch_size=32),
          epochs=50,
          validation_data=(X_test, y_test),
          callbacks=[early_stopping])

# Evaluating the model
y_pred = np.argmax(model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred, zero_division=1))
print(confusion_matrix(y_test, y_pred))

# Convert the Keras model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
tflite_model_path = '/kaggle/working/model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f'TensorFlow Lite model saved to: {tflite_model_path}')


100%|██████████| 51/51 [02:06<00:00,  2.48s/it]


Labels mapping saved to: /kaggle/working/labels.txt
Epoch 1/50
264/264 [==============================] - 111s 330ms/step - loss: 3.8314 - accuracy: 0.0693 - val_loss: 3.5191 - val_accuracy: 0.1181
Epoch 2/50
264/264 [==============================] - 82s 311ms/step - loss: 2.7366 - accuracy: 0.2490 - val_loss: 2.9114 - val_accuracy: 0.2556
Epoch 3/50
264/264 [==============================] - 82s 309ms/step - loss: 1.8051 - accuracy: 0.4753 - val_loss: 2.3963 - val_accuracy: 0.4139
Epoch 4/50
264/264 [==============================] - 83s 313ms/step - loss: 1.2039 - accuracy: 0.6432 - val_loss: 1.8715 - val_accuracy: 0.5377
Epoch 5/50
264/264 [==============================] - 81s 307ms/step - loss: 0.9175 - accuracy: 0.7236 - val_loss: 1.0293 - val_accuracy: 0.7008
Epoch 6/50
264/264 [==============================] - 82s 308ms/step - loss: 0.6986 - accuracy: 0.7862 - val_loss: 0.5901 - val_accuracy: 0.8321
Epoch 7/50
264/264 [==============================] - 81s 308ms/step - loss: 

In [ ]:
import numpy as np
import os
import cv2
import random
import matplotlib.pyplot as plt

# Load and preprocess the data (224x224 resolution with enhanced data augmentation)
folder_path = '/kaggle/input/bangla-handwritten-digit/total image 1-50'
datagen, data, labels_mapping = create_data(folder_path, target_size=(224, 224))
X = np.array([i[0] for i in data])
y = np.array([labels_mapping[i[1]] for i in data])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Evaluate the model on random images from the test set
num_images_to_check = 10

for _ in range(num_images_to_check):
    # Choose a random index from the test set
    random_index = random.randint(0, len(X_test) - 1)

    # Select a random image and its true class label
    random_image = X_test[random_index]
    true_label = y_test[random_index]

    # Reshape the image to fit the model input shape
    input_image = random_image.reshape(1, 224, 224, 3)

    # Make a prediction using the model
    predicted_probs = model.predict(input_image)
    predicted_label = np.argmax(predicted_probs)

    # Get the class names from the labels_mapping dictionary
    class_names = {v: k for k, v in labels_mapping.items()}

    # Display the true and predicted classes
    print(f"True Class: {class_names[true_label]}")
    print(f"Predicted Class: {class_names[predicted_label]}")

    # Display the predicted probabilities for each class
    for i, prob in enumerate(predicted_probs[0]):
        print(f"Probability for class {class_names[i]}: {prob:.4f}")

    # Display the image
    plt.imshow(random_image)
    plt.title(f"True: {class_names[true_label]}, Predicted: {class_names[predicted_label]}")
    plt.show()
